In [1]:
from mysql.connector import connect
import pandas as pd

In [19]:
conn = connect(
    host='db',
    database='sakila',
    username='root',
    password='root_pass'
)

In [34]:
# Lets try query top grossing movie rating

origin_sql = '''
SELECT
    rating,
    sum(amount) as total_sale
From(
        Select
            f.rating,
            P.amount
        From
            payment P
            INNER JOIN rental r ON (P.rental_id = r.rental_id)
            INNER JOIN inventory I ON (I.inventory_id = r.inventory_id)
            INNER JOIN film f ON (I.film_id = f.film_id)
    ) as L
GROUP BY
    rating
ORDER BY
    total_sale DESC
'''

star_sql = '''
SELECT
    rating,
    sum(sales_amount) as total_sale
FROM
    fact_sales fs
    INNER JOIN dimMovie dm ON (fs.movie_key = dm.movie_key)
GROUP BY
    rating
ORDER BY
    total_sale DESC
'''

In [41]:
%%timeit
pd.read_sql(origin_sql, conn)

69.6 ms ± 10.9 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [42]:
%%timeit
pd.read_sql(star_sql, conn)

31.4 ms ± 7.97 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [43]:
# Lets see the result of the query for origin database
pd.read_sql(origin_sql, conn)

,rating,total_sale
0,PG-13,15259.16
1,NC-17,13875.07
2,PG,13337.91
3,R,13270.19
4,G,11664.23


In [44]:
# Now for star schema
pd.read_sql(star_sql, conn)

,rating,total_sale
0,PG-13,15259.159616
1,NC-17,13875.069640
2,PG,13337.909667
3,R,13270.189666
4,G,11664.229721


With only 10 loop on simple query in we already see the diference on performance between Original database and Star Schema, Query on Star Schema run 2 times faster than on the origin Database as the Star schema need fewer join the achieve same result. 